# **Guardrails.AI**  

## **Custom parser for LLM response**

- LLMs sometimes tend to hallucinate and may not provide accurate structure

- Generating correct LLM outputs helps with improving their reliability


- While refining the prompt to improve accuracy might be one apporach, a
  stochastic system such as an LLM,  would still need more **systematic** ways to refine and validate the output.

- Guardrail mechanisms make the response from LLM robust, by enforcing a **structure**, **type** and **improving quality**

- Guardrails may also take **corrective actions** when validation fails


# **Installing Dependencies**

In [1]:
!pip install guardrails-ai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.8/114.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.2/106.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.2.1
    Uninstalling pydantic-2.2.1:
      Successfully uninstalled pydantic-2.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install openai
import openai

- **RAIL - Reliable AI Markup Language**  helps provide the output structure required to refine the response from LLM, specify prompts, dynamic inputs





In [90]:
rail_spec = """
<rail version="0.1">
<output>
<object name="patient_info">
<string name="gender" description="Patient's gender" />
<integer name="age" format="valid-range: 0 100" />
<string name="symptoms" description="Symptoms that the patient is currently experiencing" />
</object>
</output>
<prompt>

Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's information.

${doctors_notes}
${gr.complete_json_suffix_v2}

</prompt>
</rail>

"""




# Create a **guardrail output parser**

In [99]:
import os
os.environ["OPENAI_API_KEY"] = "sk--"

In [100]:
from rich import print

from langchain.output_parsers import GuardrailsOutputParser

In [101]:
from langchain.prompts import PromptTemplate

In [102]:
from langchain.llms import OpenAI

In [103]:
output_parser = GuardrailsOutputParser.from_rail_string(rail_spec)

In [104]:
print(output_parser.guard.base_prompt)

Given the following doctor's notes about a patient, please extract a dictionary that contains the patient's 
information.

${doctors_notes}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <object name="patient_info">
        <string description="Patient's gender" name="gender"/>
        <integer format="valid-range: min=0 max=100" name="age"/>
        <string description="Symptoms that the patient is currently experiencing" name="symptoms"/>
    </object>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

# **Creating a prompt template**

In [106]:
prompt = PromptTemplate(
    template=output_parser.guard.prompt.escape(),
    input_variables=output_parser.guard.prompt.variable_names,
)

In [113]:
model = OpenAI(temperature = 0)

doctors_notes ="""
49 y/o Male with mild rash to face &amp; hair, in beard, eyebrows &knee; nares.
Itchy, flaky, slightly scaly. Moderate response to OTC steroid cream
"""
output = model(prompt.format_prompt(doctors_notes=doctors_notes).to_string())

In [114]:
print(output_parser.parse(output))

{
    'patient_info': {
        'gender': 'Male',
        'age': 49,
        'symptoms': 'Mild rash to face & hair, in beard, eyebrows & knee; nares. Itchy, flaky, slightly scaly. 
Moderate response to OTC steroid cream'
    }
}